In [21]:
import numpy as np
from keras.utils import to_categorical
from torch.ao.nn.quantized import Softmax
from torch.ao.quantization.utils import activation_is_int8_quantized

In [1]:
alphabets='ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [2]:
list(enumerate(alphabets))

[(0, 'A'),
 (1, 'B'),
 (2, 'C'),
 (3, 'D'),
 (4, 'E'),
 (5, 'F'),
 (6, 'G'),
 (7, 'H'),
 (8, 'I'),
 (9, 'J'),
 (10, 'K'),
 (11, 'L'),
 (12, 'M'),
 (13, 'N'),
 (14, 'O'),
 (15, 'P'),
 (16, 'Q'),
 (17, 'R'),
 (18, 'S'),
 (19, 'T'),
 (20, 'U'),
 (21, 'V'),
 (22, 'W'),
 (23, 'X'),
 (24, 'Y'),
 (25, 'Z')]

In [3]:
int_to_char=dict(list(enumerate(alphabets)))

In [4]:
int_to_char[6]

'G'

In [6]:
char_to_int = dict((v, i) for i, v in enumerate(alphabets))
char_to_int

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [7]:
char_to_int['X']

23

Prepare the dataset

In [12]:
seq_length=1
x=[]
y=[]

for i in range(len(alphabets)-seq_length):
    seq_in=[alphabets[i]]
    seq_out=alphabets[i+1]
    print(seq_in,'-->',seq_out)
    x+=[[char_to_int[seq_in[0]]]]
    y+=[char_to_int[seq_out]]

['A'] --> B
['B'] --> C
['C'] --> D
['D'] --> E
['E'] --> F
['F'] --> G
['G'] --> H
['H'] --> I
['I'] --> J
['J'] --> K
['K'] --> L
['L'] --> M
['M'] --> N
['N'] --> O
['O'] --> P
['P'] --> Q
['Q'] --> R
['R'] --> S
['S'] --> T
['T'] --> U
['U'] --> V
['V'] --> W
['W'] --> X
['X'] --> Y
['Y'] --> Z


In [13]:
x,y

([[0],
  [1],
  [2],
  [3],
  [4],
  [5],
  [6],
  [7],
  [8],
  [9],
  [10],
  [11],
  [12],
  [13],
  [14],
  [15],
  [16],
  [17],
  [18],
  [19],
  [20],
  [21],
  [22],
  [23],
  [24]],
 [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25])

In [15]:
x=np.reshape(x,(25,1,1))

In [16]:
x

array([[[ 0]],

       [[ 1]],

       [[ 2]],

       [[ 3]],

       [[ 4]],

       [[ 5]],

       [[ 6]],

       [[ 7]],

       [[ 8]],

       [[ 9]],

       [[10]],

       [[11]],

       [[12]],

       [[13]],

       [[14]],

       [[15]],

       [[16]],

       [[17]],

       [[18]],

       [[19]],

       [[20]],

       [[21]],

       [[22]],

       [[23]],

       [[24]]])

In [17]:
x_norm=x/25

In [47]:
x_norm.shape

(25, 1, 1)

In [19]:
a=[1,3,2,3,2,1,1,2,3]

In [22]:
to_categorical(a)

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [23]:
y_new=to_categorical(y)
y_new

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

Build the model

In [24]:
from keras.layers import SimpleRNN, Dense, Input
from keras.models import Sequential

In [25]:
model=Sequential()

In [26]:
model.add(Input((1,1)))
model.add(Dense(25,activation='relu'))
model.add(SimpleRNN(32))
model.add(Dense(26,activation='softmax'))

In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1, 25)          │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,764 (10.80 KB)

 Trainable params: 2,764 (10.80 KB)

 Non-trainable params: 0 (0.00 B)

Compile the model

In [28]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Train the model

In [38]:
model.fit(x_norm,y_new,epochs=500,batch_size=1)

Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7600 - loss: 1.3208
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8400 - loss: 1.3229
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8800 - loss: 1.3201
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7200 - loss: 1.3226
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8000 - loss: 1.3219
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8000 - loss: 1.3205
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8000 - loss: 1.3235
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7600 - loss: 1.3201
Epoch 9/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8400 - loss: 1.3173
Epoch 10/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8400 - loss: 1.3111
Epoch 11/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8400 - loss: 1.3158
Epoch 12/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy:

In [39]:
model.evaluate(x_norm,y_new,batch_size=1)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9200 - loss: 0.7464


[0.7464209198951721, 0.9200000166893005]

In [56]:
from numpy import argmax

new='P'
n=np.array(char_to_int[new]).reshape(1,1,1)/25
pred = model(n)
char = argmax(pred.numpy())
print(int_to_char[char])

Q


## Sequence length = 3

In [84]:
seq_length=3
x=[]
y=[]

for i in range(len(alphabets)-seq_length):
    seq_in=alphabets[i:i+seq_length]
    seq_out=alphabets[i+seq_length]
    print(seq_in,'-->',seq_out)
    x.append([char_to_int[char] for char in seq_in])
    y+=[char_to_int[seq_out]]

ABC --> D
BCD --> E
CDE --> F
DEF --> G
EFG --> H
FGH --> I
GHI --> J
HIJ --> K
IJK --> L
JKL --> M
KLM --> N
LMN --> O
MNO --> P
NOP --> Q
OPQ --> R
PQR --> S
QRS --> T
RST --> U
STU --> V
TUV --> W
UVW --> X
VWX --> Y
WXY --> Z


In [82]:
print(x)

[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [9, 10, 11], [10, 11, 12], [11, 12, 13], [12, 13, 14], [13, 14, 15], [14, 15, 16], [15, 16, 17], [16, 17, 18], [17, 18, 19], [18, 19, 20], [19, 20, 21], [20, 21, 22], [21, 22, 23], [22, 23, 24]]


In [85]:
x=np.reshape(x,(23,3,1))
x

array([[[ 0],
        [ 1],
        [ 2]],

       [[ 1],
        [ 2],
        [ 3]],

       [[ 2],
        [ 3],
        [ 4]],

       [[ 3],
        [ 4],
        [ 5]],

       [[ 4],
        [ 5],
        [ 6]],

       [[ 5],
        [ 6],
        [ 7]],

       [[ 6],
        [ 7],
        [ 8]],

       [[ 7],
        [ 8],
        [ 9]],

       [[ 8],
        [ 9],
        [10]],

       [[ 9],
        [10],
        [11]],

       [[10],
        [11],
        [12]],

       [[11],
        [12],
        [13]],

       [[12],
        [13],
        [14]],

       [[13],
        [14],
        [15]],

       [[14],
        [15],
        [16]],

       [[15],
        [16],
        [17]],

       [[16],
        [17],
        [18]],

       [[17],
        [18],
        [19]],

       [[18],
        [19],
        [20]],

       [[19],
        [20],
        [21]],

       [[20],
        [21],
        [22]],

       [[21],
        [22],
        [23]],

       [[22],
        [23],
    

In [86]:
x_norm = x/25
y_new = to_categorical(y)
y_new

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

In [68]:
model = Sequential()
model.add(Input((3, 1)))
model.add(Dense(23, activation='relu'))
model.add(SimpleRNN(32))
model.add(Dense(26, activation='softmax'))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 3, 23)          │            46 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 32)             │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,696 (10.53 KB)

 Trainable params: 2,696 (10.53 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [70]:
model.fit(x_norm,y_new,epochs=500,batch_size=1)

Epoch 1/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0000e+00 - loss: 3.2866
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 3.2388
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.2098
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.1816
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.1534
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0435 - loss: 3.1274
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.0969
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0870 - loss: 3.0643
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1304 - loss: 3.0270
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0870 - loss: 2.9774
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0435 - loss: 2.9306    
Epoch 12/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

In [71]:
model.evaluate(x_norm, y_new, batch_size=1)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9565 - loss: 0.2562


[0.2561889588832855, 0.95652174949646]

In [92]:
string = "ABC"
new = [char_to_int[char] for char in string]
n = np.array(new).reshape(1, 3, 1) / 25
pred = model(n)
char = argmax(pred.numpy())
print(string, "-->", int_to_char[char])

ABC --> D
